In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/03/01 20:17:16] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/01 20:17:16] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/01 20:17:16] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [13]:
gui = GUI('data/input/temp.jpg')
gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui.load_detection_result()
gui.show_detection_result()

[2022/03/01 20:18:52] root DEBUG: dt_boxes num : 17, elapse : 0.13954925537109375
[2022/03/01 20:18:52] root DEBUG: cls num  : 17, elapse : 0.06224226951599121
[2022/03/01 20:18:52] root DEBUG: rec_res num  : 17, elapse : 0.12114596366882324
[Text Detection Completed in 0.343 s] Input: data/input/temp.jpg Output: data/output/ocr/temp.json
[Compo Detection Completed in 0.400 s] Input: data/input/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/input/temp.jpg Output: data/output/merge/temp.jpg


In [14]:
# load operations
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.shrink_target_element_img()
    op.resize(int(op.ui_img_width * (800 / op.ui_img_height)), 800)        
    op.detect_text(paddle_ocr)
    operations.append(op)
#     op.show_target_ele()
    break

[2022/03/01 20:18:55] root DEBUG: dt_boxes num : 0, elapse : 0.0036602020263671875
[2022/03/01 20:18:55] root DEBUG: cls num  : 0, elapse : 0
[2022/03/01 20:18:55] root DEBUG: rec_res num  : 0, elapse : 1.6689300537109375e-06
[Text Detection Completed in 0.006 s] Input: data/operation/target.png Output: data/operation/target.json


In [15]:
opr = operations[0]
matched_e = gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model, show=True)

2 0.7920485
3 0.31549275
6 0.32304513
7 0.25114357
11 0.329207
12 0.1678816
13 0.5638797
Match by image


In [13]:
opr.show_target_ele()

In [7]:
gui.adjust_elements_by_screen()

In [8]:
b = gui.draw_element_after_screen_conversion()

In [16]:
matched_e.clip.shape

(12, 18, 3)

In [12]:
opr.target_element_img.shape

(14, 19, 3)